In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as BS
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'}

In [2]:
# 1: 'see all buying options'
# 2: 'Other Sellers on Amazon'
# 3: 'New (x) from $y.00' OR 'Used (x) from $y.00'
# 4 : Available from these sellers.

links = ['https://www.amazon.com/s?k=nintendo+switch+console&page=', 
]
"""
'https://www.amazon.com/s?k=nintendo+switch+console&page=',
'https://www.amazon.com/s?k=robot+vacuum+cleaner&page=',
'https://www.amazon.com/s?k=apple+iphone+11+pro%2C+256gb&rh=p_89%3AApple&dc&page=',
'https://www.amazon.com/s?k=aorus+geforce+rtx+2070&page='"""

df_main = pd.DataFrame(columns = ['Price', 'Condition', 'Seller', 'ProductName', 'Brand',
                                 'Feature1Name', 'Feature1', 'Feature2Name', 'Feature2'])

for link in links:
    for pn in range(1,2):
        url_main = link + str(pn)
        print(url_main)
        response_main=requests.get(url_main, headers=headers)
        soup_main=BS(response_main.content, features="lxml")
        a = soup_main.select('a.a-link-normal.a-text-normal')
        for i in a:
            url_link = 'https://www.amazon.com' + i['href']
            print(url_link)
            try:
                response_link=requests.get(url_link, headers=headers)
                soup_link=BS(response_link.content, features="lxml")
                source_codes = ['a#buybox-see-all-buying-choices-announce.a-button-text', 
                                '#olpDiv a', 
                                'a.a-touch-link.a-box.olp-touch-link', 
                                '#availability a', 
                                '#olpLinkWidget_feature_div .a-box-inner',
                               '#olpLinkWidget_feature_div .a-color-price',
                               '#olp-upd-new-used .a-link-normal']
                f = 0
                for s in source_codes:
                    try:
                        b = soup_link.select(s)
                        if b != []:
                            f +=1
                            for i in b:
                                url_link_in_link = 'https://www.amazon.com' + i['href']
                                try:
                                    response_link_in_link=requests.get(url_link_in_link, headers=headers)
                                    soup_link_in_link=BS(response_link_in_link.content, features="lxml")
                                    response_link_in_link_in_link=requests.get(url_link_in_link, headers=headers)
                                    soup=BS(response_link_in_link_in_link.content, features="lxml")
                                    print('there are links in link')
                                    q = soup.select('ul.a-unordered-list.a-nostyle.a-button-list.a-declarative.a-button-toggle-group.a-horizontal.a-spacing-small')
                                    list_f1 = []
                                    for i in q[0]:
                                        for j in i:
                                            for k in j:
                                                for l in k:
                                                    for p in l:
                                                            list_f1.append(p)
                                    url_feature_1 = []
                                    for i in list_f1:
                                        try:
                                            url_feature_1.append(i['href'])
                                        except:
                                            pass 
                                    for i in url_feature_1:
                                        url_detay = 'https://www.amazon.com' + i
                                        try:
                                            response_detay=requests.get(url_detay, headers=headers)
                                            soup_detay=BS(response_detay.content, features="lxml")
                                            q = soup_detay.select('ul.a-unordered-list.a-nostyle.a-button-list.a-declarative.a-button-toggle-group.a-horizontal.a-spacing-small')
                                            list_f2 = []
                                            for n in q[1]:
                                                for j in n:
                                                    for k in j:
                                                        for l in k:
                                                            for p in l:
                                                                    list_f2.append(p)
                                                                    url_feature_2 = []
                                                                    for i in list_f2:
                                                                        try:
                                                                            url_feature_2.append(i['href'])
                                                                        except:
                                                                            pass
                                            for i in url_feature_2:
                                                url_alt = 'https://www.amazon.com' + i
                                                
                                                price_list = []
                                                condition_list = []
                                                seller_list = []
                                                for i in range(1,2):
                                                    url = url_alt[:51] + 'ref=olp_page_{}?ie=UTF8&f_all=true&startIndex={}0'.format(str(i), str(i-1))
                                                    response=requests.get(url, headers=headers)
                                                    soup=BS(response.content, features="lxml") 
                                                    child_list = [3,5,7,9,12,14,16,18,20,22] 
                                                    for i in child_list:
                                                        try:
                                                            price_list.append(soup.select('.olpOffer:nth-child({}) .olpOfferPrice'.format(str(i)))[0].get_text().strip())
                                                        except:
                                                            price_list.append(np.nan)  
                                                        try:
                                                            condition_list.append(soup.select('.olpOffer:nth-child({}) .olpCondition'.format(str(i)))[0].get_text().strip())
                                                        except:
                                                            condition_list.append(np.nan)
                                                        try:
                                                            seller_list.append(soup.select('.olpOffer:nth-child({}) .a-text-bold a'.format(str(i)))[0].get_text().strip())
                                                        except:
                                                            seller_list.append(np.nan)
                                                            
                                                df = pd.DataFrame(list(zip(price_list, condition_list, seller_list)), columns= ['Price', 'Condition', 'Seller']) 

                                                try:
                                                    product_name=soup.select("#olpProductDetails .a-spacing-none")[0].get_text().strip()
                                                except:
                                                    product_name = np.nan
                                                df['ProductName'] = product_name                                                
                                                try:
                                                    brand = soup.select('#olpProductByline')[0].get_text().strip()
                                                except:
                                                    brand = np.nan
                                                df['Brand'] = brand

                                                try:
                                                    Feature1Name = soup.select('.a-spacing-micro:nth-child(1) .a-size-base:nth-child(1)')[0].get_text().strip()
                                                    Feature1 = soup.select('.a-spacing-micro:nth-child(1) .a-text-bold')[0].get_text().strip()
                                                except:
                                                    Feature1Name = np.nan
                                                    Feature1 = np.nan
                                                df['Feature1Name'] = Feature1Name
                                                df['Feature1'] = Feature1

                                                try:
                                                    Feature2Name = soup.select('.a-spacing-small+ .a-spacing-micro .a-size-base:nth-child(1)')[0].get_text().strip()
                                                    Feature2 = soup.select('.a-spacing-small+ .a-spacing-micro .a-text-bold')[0].get_text().strip()
                                                except:
                                                    Feature2Name = np.nan
                                                    Feature2 = np.nan
                                                df['Feature2Name'] = Feature2Name
                                                df['Feature2'] = Feature2                                                            

                                                df_main = df_main.append(df)   
                                                
                                                print(url_alt[:51], 'url_double_feature')
                                                
                                        except:
                                            print(url_detay[:51], 'url_single_feature')
                                except:
                                    print(url_link_in_link[:51], 'url_no_feature')
                        else:
                            f = f
                    except:
                        pass
                if f == 0:
                    print('one seller')
            except:
                pass

https://www.amazon.com/s?k=nintendo+switch+console&page=1
https://www.amazon.com/Nintendo-Switch-Neon-Blue-Joy%E2%80%91/dp/B07VGRJDFY/ref=sr_1_1?keywords=nintendo+switch+console&qid=1583081405&sr=8-1
there are links in link
https://www.amazon.com/gp/offer-listing/B07VJRZ62R/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VGRJDFY/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YBVW7RH/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YBVJRD7/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YMJ4XBP/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YMPLCXM/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YMM3RHZ/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YMR6NB8/ url_double_fea

https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/Nintendo-Switch-Lite-Gray/dp/B07V2BBMK4/ref=sr_1_3?keywords=nintendo+switch+console&qid=1583081405&sr=8-3
there are links in link
https://www.amazon.com/gp/offer-listing/B084Y3VVNG/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V2BBMK4/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VHFHHY5/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VKJD6WT/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VKJ57TW/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_fea

https://www.amazon.com/gp/offer-listing/B07V3G6C1F/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VHFVP1S/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VM4MZM7/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VKZRQDF/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
there are links in link
https://www.amazon.com/gp/offer-listing/B084Y3VVNG/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V2BBMK4/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VHFHHY5/ u

https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
there are links in link
https://www.amazon.com/gp/offer-listing/B07VJRZ62R/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VGRJDFY/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YBVW7RH/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YBVJRD7/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YMJ4XBP/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YMPLCXM/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YMM3RHZ/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YMR6NB8/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ u

https://www.amazon.com/gp/offer-listing/B07PW245TZ/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07PW245TZ/ url_double_feature
https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_mtf_aps_sr_pg1_1?ie=UTF8&adId=A00569521CBC17R8GWGZW&url=%2FChrome-Silver-Custom-Console-Nintendo-Switch%2Fdp%2FB07V323BFC%2Fref%3Dsr_1_9_sspa%3Fkeywords%3Dnintendo%2Bswitch%2Bconsole%26qid%3D1583081405%26sr%3D8-9-spons%26psc%3D1&qualifier=1583081405&id=4338689050938045&widgetName=sp_mtf
there are links in link
https://www.amazon.com/gp/offer-listing/B07V22H56N/ url_single_feature
https://www.amazon.com/gp/offer-listing/B07V33DWH2/ url_single_feature
https://www.amazon.com/gp/offer-listing/B07V32H539/ url_single_feature
https://www.amazon.com/gp/offer-listing/B07V32BWBC/ url_single_feature
https://www.amazon.com/gp/offer-listing/B07V323BFC/ url_single_feature
https://www.amazon.com/gp/offer-listing/B07V348J38/ url_single_feature
https://www.amazon.com/gp/offer-listing/B07V321GS4/ 

https://www.amazon.com/gp/offer-listing/B07YMR6NB8/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
there are links in link
https://www.amazon.com/gp/offer-listing/B07VJRZ62R/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VGRJDFY/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YBVW7RH/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YBVJRD7/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YMJ4XBP/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07YMPLCXM/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07XLGBYM3/ u

https://www.amazon.com/gp/offer-listing/B07VM4MZM7/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VKZRQDF/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
there are links in link
https://www.amazon.com/gp/offer-listing/B084Y3VVNG/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V3T7WF1/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07V2BBMK4/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VHFHHY5/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VKJD6WT/ url_double_feature
https://www.amazon.com/gp/offer-listing/B07VKJ57TW/ u

In [7]:
df_main = df_main.dropna()
df_main = df_main.drop_duplicates()
df_main[df_main['Condition'] == 'New']

,Price,Condition,Seller,ProductName,Brand,Feature1Name,Feature1,Feature2Name,Feature2
0,$314.00,New,ardmechanic,Nintendo Switch with Gray Joy‑Con - HAC-001(-01),by Nintendo,Style:,Console,Color:,Gray HAC-001(-01)
1,$324.00,New,RichMondHS [SN Recorded],Nintendo Switch with Gray Joy‑Con - HAC-001(-01),by Nintendo,Style:,Console,Color:,Gray HAC-001(-01)
2,$329.00,New,Ontrend Deals,Nintendo Switch with Gray Joy‑Con - HAC-001(-01),by Nintendo,Style:,Console,Color:,Gray HAC-001(-01)
3,$330.00,New,TechFamily,Nintendo Switch with Gray Joy‑Con - HAC-001(-01),by Nintendo,Style:,Console,Color:,Gray HAC-001(-01)
4,$339.99,New,Always Innovating,Nintendo Switch with Gray Joy‑Con - HAC-001(-01),by Nintendo,Style:,Console,Color:,Gray HAC-001(-01)
5,$394.01,New,Wolfhope,Nintendo Switch with Gray Joy‑Con - HAC-001(-01),by Nintendo,Style:,Console,Color:,Gray HAC-001(-01)
6,$328.05,New,Springfield Distribution,Nintendo Switch with Gray Joy‑Con - HAC-001(-01),by Nintendo,Style:,Console,Color:,Gray HAC-001(-01)
7,$399.99,New,Acebeach,Nintendo Switch with Gray Joy‑Con - HAC-001(-01),by Nintendo,Style:,Console,Color:,Gray HAC-001(-01)
0,$324.99,New,MallStop,Nintendo Switch with Neon Blue and Neon Red Jo...,by Nintendo,Style:,Console,Color:,Neon Blue and Red HAC-001(-01)
1,$327.50,New,Kliickit,Nintendo Switch with Neon Blue and Neon Red Jo...,by Nintendo,Style:,Console,Color:,Neon Blue and Red HAC-001(-01)
